In [ ]:
########이미지 5번 처리를 위함 #####################################################

In [38]:
# import sys
import random
import numpy as np
import cv2

src = cv2.imread("C:/Users/Yunji Noh/OneDrive/image_processing/data/BC7.jpg")

if src is None:
    print('Image load failed!')
    sys.exit()

#src = cv2.resize(src, (0, 0), fx=0.5, fy=0.5)
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)


h, w = src.shape[:2]
dst1 = np.zeros((h, w, 3), np.uint8)
dst2 = np.zeros((h, w, 3), np.uint8)

# 이진화
_, src_bin = cv2.threshold(src_gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# 외곽선 검출
contours, _ = cv2.findContours(src_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

for i in range(len(contours)):
    pts = contours[i]

    c = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    cv2.drawContours(dst1, contours, i, c, 1)

    # 너무 작은 객체는 제외
    if (cv2.contourArea(pts) < 1000):
        continue

    # 외곽선 근사화
    approx = cv2.approxPolyDP(pts, cv2.arcLength(pts, True)*0.02, True)

    # 컨벡스가 아니면 제외
    if not cv2.isContourConvex(approx):
        continue

    if len(approx) == 4:
        cv2.drawContours(dst2, contours, i, c, 2)
        break  # 첫 번째로 검출된 외곽선만 사용하여 perspective transformation 수행

        
if len(approx) == 4:
    # 사각형의 네 꼭지점 좌표 추출
    points = approx.reshape(-1, 2).astype(np.float32)
    
    # 네 꼭지점 좌표에서 최소/최대 x, y 값을 구하여 w, h 계산
    min_x = np.min(points[:, 0])
    max_x = np.max(points[:, 0])
    min_y = np.min(points[:, 1])
    max_y = np.max(points[:, 1])
    
    w = int(max_y-min_y)
    h = int(max_x-min_x)

#- min_y, - min_x

    # 변환 후 좌표 설정 (사진마다 다름)
    dst_points = np.array([[0, h], [w, h],[w, 0], [0, 0]], dtype=np.float32)
    
    # 변환 행렬 계산
    transform_matrix = cv2.getPerspectiveTransform(points, dst_points)
    
    # 이미지 변환
    dst_image = cv2.warpPerspective(src, transform_matrix, (w, h))
    sharp = cv2.filter2D(dst_image, -1, np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]]))
    
    
    
    # 결과 출력
    cv2.imshow('dst_image', dst_image)
   # cv2.imshow('src', src)
    #cv2.imshow('src_bin', src_bin)
    #cv2.imshow('dst1', dst1)
    cv2.imshow('dst2', dst2)
    cv2.imshow('sharp', sharp)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
#####################################################################################################

In [57]:
#아래 21번 이미지 처리 가능 (5번은 다른 알고리즘 썻음)
#되는 번호: 1,2,3,8,9,10,11,21
#안되는 번호(4, 6,7, 12,13,14,15, 16,17,18, 19, 20)

import random
import numpy as np
import cv2

src = cv2.imread("C:/Users/Yunji Noh/OneDrive/image_processing/data/BC21.jpg", cv2.IMREAD_GRAYSCALE)

# Noise removal with Gaussian blur
noise_removed = cv2.GaussianBlur(src, (9, 9), 0)

# Adaptive thresholding
src_bin = cv2.adaptiveThreshold(noise_removed, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

# Edge detection with Canny
edges = cv2.Canny(src_bin, 50, 100)

# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter contours based on area
min_contour_area = 1000  # Adjust this threshold as needed
filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]

# Draw contours on the original image
contour_src = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)
cv2.drawContours(contour_src, filtered_contours, -1, (0, 255, 255), 2)

# Corner detection in contour regions
corners = []
for contour in filtered_contours:
    # Approximate contour as a polygon
    epsilon = 0.1 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)
    
    # Corner detection in the polygon
    for corner in approx:
        corners.append(corner[0])
        cv2.circle(contour_src, tuple(corner[0]), 3, (0, 0, 255), -1)

# Display the result
cv2.imshow('Noise Removal', noise_removed)
cv2.imshow('Binary Image', src_bin)
cv2.imshow('Edges', edges)
cv2.imshow('Contours', contour_src)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
##빛반사 해결을 위한 노력 2 (Resized )##############################################

In [58]:
import random
import numpy as np
import cv2

src = cv2.imread("C:/Users/Yunji Noh/OneDrive/image_processing/data/BC4.jpg", cv2.IMREAD_GRAYSCALE)

# Noise removal with Gaussian blur
noise_removed = cv2.GaussianBlur(src, (9, 9), 0)

# Adaptive thresholding
src_bin = cv2.adaptiveThreshold(noise_removed, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

# Edge detection with Canny
edges = cv2.Canny(src_bin, 50, 100)

# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter contours based on area
min_contour_area = 1000  # Adjust this threshold as needed
filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]

# Draw contours on the original image
contour_src = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)
cv2.drawContours(contour_src, filtered_contours, -1, (0, 255, 255), 2)

# Corner detection in contour regions
corners = []
for contour in filtered_contours:
    # Approximate contour as a polygon
    epsilon = 0.1 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)
    
    # Corner detection in the polygon
    for corner in approx:
        corners.append(corner[0])
        cv2.circle(contour_src, tuple(corner[0]), 3, (0, 0, 255), -1)

# Perform perspective transformation
if len(filtered_contours) > 0:
    for contour in filtered_contours:
        # Get the bounding rectangle of the contour
        x, y, w, h = cv2.boundingRect(contour)
        
        # Perform perspective transformation
        transformed_image = src[y:y+h, x:x+w]
        
        # Display the transformed image
        cv2.imshow('Transformed Image', transformed_image)
        cv2.waitKey(0)
else:
    print("No contours detected.")

# Display the result
cv2.imshow('Noise Removal', noise_removed)
cv2.imshow('Binary Image', src_bin)
cv2.imshow('Edges', edges)
cv2.imshow('Contours', contour_src)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
#########지금 실행중인 부분########################################################
#되는 번호: 1,2,3,5,8,9,10,11,21 (시발발)

#X: 4, 6, 7, 

In [56]:
import random
import numpy as np
import cv2

src = cv2.imread("C:/Users/Yunji Noh/OneDrive/image_processing/data/BC15.jpg", cv2.IMREAD_GRAYSCALE)

# Noise removal with Gaussian blur
noise_removed = cv2.GaussianBlur(src, (9, 9), 0)

# Adaptive thresholding
src_bin = cv2.adaptiveThreshold(noise_removed, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

# Edge detection with Canny
edges = cv2.Canny(src_bin, 50, 100)

# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter contours based on area
min_contour_area = 1000  # Adjust this threshold as needed
filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]

# Perform perspective transformation
if len(filtered_contours) > 0:
    # Find the convex hull of the contour
    hull = cv2.convexHull(filtered_contours[0])
    
    # Create a mask for the convex hull
    mask = np.zeros_like(src)
    cv2.drawContours(mask, [hull], 0, (255), -1)
    
    # Apply bitwise AND operation to extract the region of interest
    roi = cv2.bitwise_and(src, mask)
    
    # Find the bounding rectangle of the contour
    x, y, w, h = cv2.boundingRect(hull)
    
    # Apply perspective transformation to the ROI
    target_points = np.float32([[0, 0], [w-1, 0], [w-1, h-1], [0, h-1]])
    source_points = np.float32([[x, y], [x+w-1, y], [x+w-1, y+h-1], [x, y+h-1]])
    transformation_matrix = cv2.getPerspectiveTransform(source_points, target_points)
    transformed_roi = cv2.warpPerspective(roi, transformation_matrix, (w, h))
    
    # Create a blank image with the same size as the original image
    transformed_image = np.zeros_like(src)
    
    # Place the transformed ROI back into the blank image
    transformed_image[y:y+h, x:x+w] = transformed_roi
    
    # Display the transformed image
    cv2.imshow('Transformed Image', transformed_image)
    cv2.waitKey(0)
else:
    print("No contours detected.")

# Display the result
#cv2.imshow('src_bin', edges)
#cv2.imshow('Noise Removal', noise_removed)
#cv2.imshow('Binary Image', src_bin)
#cv2.imshow('Edges', edges)
cv2.waitKey(0)
cv2.destroyAllWindows()
